In [7]:
page = Page(page_id=1)
page.insert_record("Hello, world!")
page.insert_record("This is HyperDB.")
page.get_records()


['Hello, world!', 'This is HyperDB.']

In [8]:
from minidb.storage.page import Page
from minidb.storage.filemanager import FileManager

page = Page(page_id=1)
page.insert_record("First record")
page.insert_record("Second record")

fm = FileManager()
fm.save_page(page)

loaded = fm.load_page(1)
loaded.get_records()



['First record', 'Second record']

In [1]:
from minidb.storage.page import Page
from minidb.buffer.buffer_manager import BufferManager

bm = BufferManager(capacity=2)

# Create and insert pages
for i in range(1, 5):
    page = Page(page_id=i)
    page.insert_record(f"Record in page {i}")
    bm.insert_page(page)

# Access a page
p = bm.get_page(2)
print(p.get_records())


Evicted page 1 from buffer.
Evicted page 2 from buffer.
Evicted page 3 from buffer.
['Record in page 2']


In [2]:
from minidb.index.index import HashIndex
from minidb.buffer.buffer_manager import BufferManager
from minidb.storage.page import Page

# Setup
bm = BufferManager(capacity=3)
index = HashIndex()

# Insert pages and index keys
for i in range(1, 4):
    page = Page(page_id=i)
    page.insert_record(f"Data for key_{i}")
    bm.insert_page(page)
    index.insert(f"key_{i}", i)

# Lookup and retrieve
key = "key_2"
page_id = index.lookup(key)
page = bm.get_page(page_id)
print(f"Lookup {key} → Page {page_id} → Records: {page.get_records()}")


Lookup key_2 → Page 2 → Records: ['Data for key_2']


In [1]:
from minidb.query.query_engine import QueryEngine
from minidb.buffer.buffer_manager import BufferManager
from minidb.storage.page import Page

bm = BufferManager(capacity=3)
qe = QueryEngine(buffer_manager=bm)

for i in range(1, 4):
    page = Page(page_id=i)
    page.insert_record(f"user_{i}: email_{i}@example.com")
    page.insert_record(f"user_{i}: phone_{i}")
    bm.insert_page(page)

scanned = qe.scan([1, 2, 3])
filtered = qe.filter(scanned, "email")
projected = qe.project(filtered)

print("Query Results:", projected)


Query Results: ['user_1: email_1@example.com', 'user_2: email_2@example.com', 'user_3: email_3@example.com']


In [18]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from minidb.buffer.buffer_manager import BufferManager

bm = BufferManager()
bm.insert_record("user_1", "alice@example.com")
print(bm.get_record("user_1"))



user_1: alice@example.com


In [2]:
for page in bm.buffer.values():
    print(f"Page {page.page_id}:", page.get_records())


Page 1: ['user_1: alice@example.com']


In [3]:
bm.delete_record("user_1")
print("After deletion:", bm.get_record("user_1"))  # Should print None

# View active records in Page 1
for page in bm.buffer.values():
    print(f"Page {page.page_id}:", page.get_records())


After deletion: None
Page 1: []
